# update campground status in google sheets

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import gspread
import json
from oauth2client import file, client, tools

In [2]:
def open_camping_sheet(key):
    """Opens the google sheet object"""
    # Setup the Sheets API
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    store = file.Storage('credentials.json')
    creds = store.get()
    if not creds or creds.invalid:
        raise RuntimeError('Need to refresh credentials')
        # This code will open an oauth session in a browser to log in
        flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
        creds = tools.run_flow(flow, store)
        
    f = gspread.authorize(creds) # authenticate with Google
    sheet = f.open_by_key(key).sheet1
    return sheet

In [3]:
def update_sheet(sheet, campground_name, status, status_col=1):
    """Updates the appropriate cell with campground status"""
    try:
        cell = sheet.find(re.compile('(?i){}'.format(campground_name)))
    except Exception as e:
        if type(e).__name__ == 'CellNotFound':
            print '{} not found in sheet'.format(campground_name)
            return None
    row = cell.row
    current_status = sheet.cell(row, status_col).value
    sheet.update_cell(row, status_col, status)

In [4]:
def get_campground_status(url):
    """Gets campground status from the campground webpage"""
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for i in soup.find_all('strong'):
        if 'Area Status: ' in i.contents:
            return i.next_sibling.strip()

In [5]:
# Google sheet key
# for sheet https://docs.google.com/spreadsheets/d/19TrtOtNcBHffXP1NFfz_XB_7xb3LbexpjVSGjyKpHWo/edit#gid=0
key = "19TrtOtNcBHffXP1NFfz_XB_7xb3LbexpjVSGjyKpHWo"
sheet = open_camping_sheet(key)

In [6]:
# NFS website(s)
forest_urls = [
    'https://www.fs.usda.gov/activity/eldorado/recreation/camping-cabins/?recid=71008&actid=29' # El Dorado
]

In [7]:
def main():
    for furl in forest_urls:
        r = requests.get(furl)
        soup=BeautifulSoup(r.text, 'html.parser')
        campgrounds = []
        url_pref = 'https://www.fs.usda.gov'
        for i in soup.find_all(re.compile("h\d")):
            if 'Campground Camping Areas' in i.contents:
                for j in i.find_next_siblings('ul'):
                    for k in j.findAll('a'):
                        url = k.get('href')
                        if not url.endswith('.pdf') and url is not None:
                            url = url_pref + url
                            status = get_campground_status(url)
                            campname = k.getText().split(' Campground')[0]
                            update_sheet(sheet, campname, status)